In [ ]:
# Repetir el proceso tras el reinicio del entorno

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

# Cargar dataset
df = pd.read_csv("C:/Users/Usuario/Documents/TFM/LFENDESA_UnifiedData.csv", sep=';')
df = df.dropna(subset=['Valoracion'])

# Crear variable multiclase (terciles)
df['Rendimiento_Nivel'] = pd.qcut(df['Valoracion'], q=3, labels=["Bajo", "Medio", "Alto"])

# Selección de características
features = [
    'Minutos', 'Puntos', 'T2 Anotados', 'T2 Lanzados', '% T2',
    'T3 Anotados', 'T3 Lanzados', '% T3', 'T1 Anotados', 'T1 Lanzados',
    'Reb. Ofensivos', 'Reb. Defensivos', 'Rebotes', 'Asistencias', 'Robos',
    'Perdidas', 'Tapones', 'Faltas Cometidas', 'Faltas Recibidas'
]

# Preparar datos
X = df[features].dropna()
y = df.loc[X.index, 'Rendimiento_Nivel']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos
tree_clf = DecisionTreeClassifier(random_state=42)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
mlp_clf = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42))
svm_clf = make_pipeline(StandardScaler(), SVC(kernel='linear', decision_function_shape='ovr', random_state=42))

# Entrenamiento
tree_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)

# Resultados
tree_report = classification_report(y_test, tree_clf.predict(X_test), output_dict=True)
rf_report = classification_report(y_test, rf_clf.predict(X_test), output_dict=True)
mlp_report = classification_report(y_test, mlp_clf.predict(X_test), output_dict=True)
svm_report = classification_report(y_test, svm_clf.predict(X_test), output_dict=True)



{'Alto': {'precision': 0.861878453038674,
  'recall': 0.8813559322033898,
  'f1-score': 0.8715083798882681,
  'support': 354.0},
 'Bajo': {'precision': 0.8877005347593583,
  'recall': 0.9095890410958904,
  'f1-score': 0.8985115020297699,
  'support': 365.0},
 'Medio': {'precision': 0.7875354107648725,
  'recall': 0.7513513513513513,
  'f1-score': 0.7690179806362379,
  'support': 370.0},
 'accuracy': 0.8466483011937558,
 'macro avg': {'precision': 0.8457047995209682,
  'recall': 0.8474321082168772,
  'f1-score': 0.8463459541847587,
  'support': 1089.0},
 'weighted avg': {'precision': 0.8452743522000544,
  'recall': 0.8466483011937558,
  'f1-score': 0.8457367470676961,
  'support': 1089.0}}

In [6]:
df = pd.DataFrame(tree_report)
df

,Alto,Bajo,Medio,accuracy,macro avg,weighted avg
precision,0.861878,0.887701,0.787535,0.846648,0.845705,0.845274
recall,0.881356,0.909589,0.751351,0.846648,0.847432,0.846648
f1-score,0.871508,0.898512,0.769018,0.846648,0.846346,0.845737
support,354.000000,365.000000,370.000000,0.846648,1089.000000,1089.000000


In [7]:
df = pd.DataFrame(rf_report)
df

,Alto,Bajo,Medio,accuracy,macro avg,weighted avg
precision,0.912921,0.925287,0.812987,0.881543,0.883732,0.883112
recall,0.918079,0.882192,0.845946,0.881543,0.882072,0.881543
f1-score,0.915493,0.903226,0.829139,0.881543,0.882619,0.882042
support,354.000000,365.000000,370.000000,0.881543,1089.000000,1089.000000


In [8]:
df = pd.DataFrame(mlp_report)
df

,Alto,Bajo,Medio,accuracy,macro avg,weighted avg
precision,0.983193,0.991643,0.967828,0.980716,0.980888,0.980805
recall,0.991525,0.975342,0.975676,0.980716,0.980848,0.980716
f1-score,0.987342,0.983425,0.971736,0.980716,0.980834,0.980727
support,354.000000,365.000000,370.000000,0.980716,1089.000000,1089.000000


In [9]:
df = pd.DataFrame(svm_report)
df

,Alto,Bajo,Medio,accuracy,macro avg,weighted avg
precision,0.983193,0.986072,0.962466,0.977043,0.977244,0.977116
recall,0.991525,0.969863,0.970270,0.977043,0.977220,0.977043
f1-score,0.987342,0.977901,0.966353,0.977043,0.977198,0.977046
support,354.000000,365.000000,370.000000,0.977043,1089.000000,1089.000000
